## General function

### Get satellite Imegary from Google Earth Engine

In [1]:
## -*- coding: utf-8 -*-
"""
Created on Sat Apr 8 14:13:00 2023
    By Saeed Rahimi
    V 1.0 Date 08/04/2023
    This madual includes several functions for downlaoding the satelite imageries from Google Earth Engine:
        Search for the imagery during the specified time-period
        Detects and removes the clouds in the scene
        Takes the median value of each pixel in all images in the stack (for the specified time-period)
        Creates a composite image
    @author: sarsh
"""

import sys
import subprocess

try:
    import ee
    print('The required modules are installed')
except ModuleNotFoundError:
    print('The required modules are NOT installed')

    # Install the required modules
    python = sys.executable
    subprocess.check_call(
        [python, '-m', 'pip', 'install', 'earthengine-api'],
        stdout=subprocess.DEVNULL
    )

## Initialize the library.
ee.Initialize()

# Configure the cloud details
CONFIGURATION = {'CLOUD_FILTER': 80, 'CLD_PRB_THRESH': 50, 'NIR_DRK_THRESH': 0.15,
                 'CLD_PRJ_DIST': 1, 'BUFFER': 50}

# Get a cloud free Sentinal2 composite from a colection between the START_DATE and END_DATE
def cld_free_sl2(START_DATE, END_DATE, AOI):
## %% Build a Sentinel-2 collection
    ## Define build a Sentinel-2 collection function
    def get_s2_sr_cld_col(start_date, end_date, aoi):
        ## Import and filter S2 SR.
        s2_sr_col = (ee.ImageCollection('COPERNICUS/S2_SR')
            .filterBounds(aoi)
            ## .filter(ee.Filter.calendarRange(START_YEAR,END_YEAR,'year'))
            ## .filter(ee.Filter.calendarRange(START_MONTH,END_MONTH,'month'))
            .filterDate(start_date, end_date)
            .filter(ee.Filter.lte('CLOUDY_PIXEL_PERCENTAGE', CLOUD_FILTER)))

        ## Import and filter s2cloudless.
        s2_cloudless_col = (ee.ImageCollection('COPERNICUS/S2_CLOUD_PROBABILITY')
            .filterBounds(aoi)
            ## .filter(ee.Filter.calendarRange(START_YEAR,END_YEAR,'year'))
            ## .filter(ee.Filter.calendarRange(START_MONTH,END_MONTH,'month'))
            .filterDate(start_date, end_date))


        ## Join the filtered s2cloudless collection to the SR collection by the 'system:index' property.
        return ee.ImageCollection(ee.Join.saveFirst('s2cloudless').apply(**{
            'primary': s2_sr_col,
            'secondary': s2_cloudless_col,
            'condition': ee.Filter.equals(**{
                'leftField': 'system:index',
                'rightField': 'system:index'
            })
        }))

## %% Cloud components
    def add_cloud_bands(img):
        ## Get s2cloudless image, subset the probability band.
        cld_prb = ee.Image(img.get('s2cloudless')).select('probability')

        ## Condition s2cloudless by the probability threshold value.
        is_cloud = cld_prb.gt(CLD_PRB_THRESH).rename('clouds')

        ## Add the cloud probability layer and cloud mask as image bands.
        return img.addBands(ee.Image([cld_prb, is_cloud]))


## %% Cloud shadow components
    def add_shadow_bands(img):
        ## Identify water pixels from the SCL band.
        not_water = img.select('SCL').neq(6)

        ## Identify dark NIR pixels that are not water (potential cloud shadow pixels).
        SR_BAND_SCALE = 1e4
        dark_pixels = img.select('B8').lt(NIR_DRK_THRESH*SR_BAND_SCALE).multiply(not_water).rename('dark_pixels')

        ## Determine the direction to project cloud shadow from clouds (assumes UTM projection).
        shadow_azimuth = ee.Number(90).subtract(ee.Number(img.get('MEAN_SOLAR_AZIMUTH_ANGLE')));

        ## Project shadows from clouds for the distance specified by the CLD_PRJ_DIST input.
        cld_proj = (img.select('clouds').directionalDistanceTransform(shadow_azimuth, CLD_PRJ_DIST*10)
            .reproject(**{'crs': img.select(0).projection(), 'scale': 100})
            .select('distance')
            .mask()
            .rename('cloud_transform'))

        ## Identify the intersection of dark pixels with cloud shadow projection.
        shadows = cld_proj.multiply(dark_pixels).rename('shadows')

        ## Add dark pixels, cloud projection, and identified shadows as image bands.
        return img.addBands(ee.Image([dark_pixels, cld_proj, shadows]))


    ## %% Final cloud-shadow mask
    def add_cld_shdw_mask(img):
        ## Add cloud component bands.
        img_cloud = add_cloud_bands(img)

        ## Add cloud shadow component bands.
        img_cloud_shadow = add_shadow_bands(img_cloud)

        ## Combine cloud and shadow mask, set cloud and shadow as value 1, else 0.
        is_cld_shdw = img_cloud_shadow.select('clouds').add(img_cloud_shadow.select('shadows')).gt(0)

        ## Remove small cloud-shadow patches and dilate remaining pixels by BUFFER input.
        ## 20 m scale is for speed, and assumes clouds don't require 10 m precision.
        is_cld_shdw = (is_cld_shdw.focal_min(2).focal_max(BUFFER*2/20)
            .reproject(**{'crs': img.select([0]).projection(), 'scale': 20})
            .rename('cloudmask'))

        ## Add the final cloud-shadow mask to the image.
        return img.addBands(is_cld_shdw)

    ## %% Define cloud mask application function
    def apply_cld_shdw_mask(img):
        ## Subset the cloudmask band and invert it so clouds/shadow are 0, else 1.
        not_cld_shdw = img.select('cloudmask').Not()

        ## Subset reflectance bands and update their masks, return the result.
        return img.select('B.*').updateMask(not_cld_shdw)

    ## Configure the cloud details
    CLOUD_FILTER = CONFIGURATION['CLOUD_FILTER']
    CLD_PRB_THRESH = CONFIGURATION['CLD_PRB_THRESH']
    NIR_DRK_THRESH = CONFIGURATION['NIR_DRK_THRESH']
    CLD_PRJ_DIST = CONFIGURATION['CLD_PRJ_DIST']
    BUFFER = CONFIGURATION['BUFFER']

    ## %% Return the cloud free single image
    s2_sr_cld_col = get_s2_sr_cld_col(START_DATE, END_DATE, AOI)
    s2_sr_cldless = (s2_sr_cld_col.filterBounds(AOI)
                                  .map(add_cld_shdw_mask)
                                  .map(apply_cld_shdw_mask)
                                  .median())
    return s2_sr_cldless

The required modules are installed


### Calculate a list of indices from a satellite image (an ee.image() object) and stack the results as an ee.image()

In [2]:
import ee

# Initialize Earth Engine
ee.Initialize()

# Function to calculate a list of indices for a specific site and return the image
def calculate_indices(img, indices, ee_polygon):        
    # Dictionary mapping index name to calculation function
    index_functions = {
        'MNDWI': lambda img: img.normalizedDifference(['B3', 'B11']).rename("MNDWI"),
        'NDVI': lambda img: img.normalizedDifference(['B8', 'B4']).rename("NDVI"),
        'NIRv': lambda img: img.normalizedDifference(['B8', 'B4']).multiply(img.select('B8')).rename("NIRv"),
        'NSMI': lambda img: img.normalizedDifference(['B8', 'B11']).rename("NSMI"),
        'BSI': lambda img: img.normalizedDifference(['B3', 'B11']).subtract(img.select('B8')).divide(img.select('B11').add(img.select('B8'))).rename("BSI"),
        'EVI': lambda img: img.expression('2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))', 
                                          {'NIR': img.select('B8'), 'RED': img.select('B4'), 'BLUE': img.select('B2')}).rename("EVI"),
        'SAVI': lambda img: img.expression('(1 + 0.5) * (NIR - RED) / (NIR + RED + 0.5)',
                                            {'NIR': img.select('B8'), 'RED': img.select('B4')}).rename("SAVI"),
        'NDMI': lambda img: img.normalizedDifference(['B8', 'B11']).rename("NDMI"),
        'NBR': lambda img: img.normalizedDifference(['B8', 'B12']).rename("NBR"),
        'CI': lambda img: img.select('B4').divide(img.select('B3')).subtract(1).rename("CI"),
        'LAI': lambda img: img.expression('3.618 * exp(-0.488 * (RED - GREEN))',
                                           {'RED': img.select('B4'), 'GREEN': img.select('B3')}).rename("LAI"),
        'FAPAR': lambda img: img.expression('(NIR - RED) / (0.88 * NIR + 0.12 * RED)',
                                             {'NIR': img.select('B8'), 'RED': img.select('B4')}).rename("FAPAR"),
    }

    # Initialize an empty ee.Image to store the calculated indices
    calc_indices = ee.Image()

    # Iterate over the list of indices and calculate each requested index in the list
    for index in indices:
        if index in index_functions:
            # Call the corresponding function to calculate the index
            index_calculation = index_functions[index](img)
            calc_indices = calc_indices.addBands(index_calculation)

    # Clip the index image to the extent of the current sampling site catchment
    calc_indices = calc_indices.clip(ee_polygon)

    return calc_indices

### Convert polygons in a shapefile to ee.FeatureCollection

In [3]:
# Import the libraries
import ee
import pyproj
from shapely.geometry import Polygon, MultiPolygon
import geopandas as gpd

# Initialize Earth Engine
ee.Initialize()

# convert a shapefile to an ee_collection
def shapefile_2_ee_collection(gdf):
    # Transform the geometry to WGS84 projection
    source_crs = "EPSG:2193"
    target_crs = "EPSG:4326"
    transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)
   
    def geometry_to_ee_polygon(geometry): 
        # Reproject the polygon to the target CRS
        try:
            # Check if the geometery is a polygon or a multipolygon type
            if isinstance(geometry, MultiPolygon):
                geometry_list = []
                for polygon in geometry.geoms:
                    # Reproject the polygon to the target CRS
                    transformed_coordinates = [transformer.transform(x, y) for x, y in polygon.exterior.coords]
                    transformed_geometry = Polygon(transformed_coordinates)
                    
                    # Create an ee.Geometry.Polygon and add it to a list
                    ee_polygon = ee.Geometry.Polygon(transformed_coordinates)
                    geometry_list += [ee_polygon]
                    
                    # Create a ee.Geometry.MultiPolygon using the list of ee.Geometry objects
                    multi_polygon = ee.Geometry.MultiPolygon(geometry_list)
                    # Dissolve the ee.Geometry.MultiPolygon to create a single ee.Geometry.Polygon
                    ee_polygon = multi_polygon.dissolve()
            
            elif isinstance(geometry, Polygon):
                # Reproject the polygon to the target CRS
                transformed_coordinates = [transformer.transform(x, y) for x, y in geometry.exterior.coords]
                transformed_geometry = Polygon(transformed_coordinates)
                ee_polygon = ee.Geometry.Polygon(transformed_coordinates)

            else:
                raise ValueError("Invalid geometry type. Must be Polygon or MultiPolygon.")
    
        except Exception as e:
            print(f"Invalid geometry at index: {e}")
        
        return ee_polygon

    # Create a list of ee.Geometry objects
    polygon_list = [geometry_to_ee_polygon(geometry) for geometry in gdf['geometry']]
    
    # Convert the list to an ee.FeatureCollection
    polygon_collection = ee.FeatureCollection(polygon_list)

    return polygon_collection

## Iterate over each capture zone (sampling point) and collect the RS indices

### Iterate over the sampling sites

In [ ]:
# import ee

# # Initialize Earth Engine
# ee.Initialize()

# # Function to perform zonal stats
# def calculate_stats(image, indices, ee_polygon):    
#     # Create an empty dictionary
#     statistics_dict = {}
    
#     # Calculate statistics for each index
#     for index in indices:
#         # Calculate multiple statistics using reduceRegion
#         stats = image.reduceRegion(
#             reducer=ee.Reducer.min().combine(
#                 reducer2=ee.Reducer.median().combine(
#                     reducer2=ee.Reducer.max().combine(
#                         reducer2=ee.Reducer.stdDev(),
#                         sharedInputs=True
#                     ),
#                     sharedInputs=True
#                 ),
#                 sharedInputs=True
#             ),
#             geometry=ee_polygon,
#             scale=10,
#             maxPixels=1e19
#         )
        
#         # Add the statistics to the dictionary
#         statistics_dict[index] = {
#           # 'min': stats.get(f'{index}_min').getInfo(),
#           'median': stats.get(f'{index}_median').getInfo(),
#           # 'max': stats.get(f'{index}_max').getInfo(),
#           'stdv': stats.get(f'{index}_stdDev').getInfo()
#         }
        
#     return statistics_dict

# # Function to convert the geometry to an ee.polygon and transform the projection
# def geometry_to_ee_polygon(geometry):
#         # Transform the geometry to WGS84 projection
#         source_crs = "EPSG:2193"
#         target_crs = "EPSG:4326"
#         transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)
#         geometry = site['geometry']
    
#         # Reproject the polygon to the target CRS
#         try:
#             # Check if the geometery is a polygon or a multipolygon type
#             if isinstance(geometry, MultiPolygon):
#                 geometry_list = []
#                 for polygon in geometry.geoms:
#                     # Reproject the polygon to the target CRS
#                     transformed_coordinates = [transformer.transform(x, y) for x, y in polygon.exterior.coords]
#                     transformed_geometry = Polygon(transformed_coordinates)
                    
#                     # Create an ee.Geometry.Polygon and add it to a list
#                     ee_polygon = ee.Geometry.Polygon(transformed_coordinates)
#                     geometry_list += [ee_polygon]
                    
#                     # Create a ee.Geometry.MultiPolygon using the list of ee.Geometry objects
#                     multi_polygon = ee.Geometry.MultiPolygon(geometry_list)
#                     # Dissolve the ee.Geometry.MultiPolygon to create a single ee.Geometry.Polygon
#                     ee_polygon = multi_polygon.dissolve()
            
#             elif isinstance(geometry, Polygon):
#                 # Reproject the polygon to the target CRS
#                 transformed_coordinates = [transformer.transform(x, y) for x, y in geometry.exterior.coords]
#                 transformed_geometry = Polygon(transformed_coordinates)
#                 ee_polygon = ee.Geometry.Polygon(transformed_coordinates)
        
#             else:
#                 raise ValueError("Invalid geometry type. Must be Polygon or MultiPolygon.")
        
#         except Exception as e:
#             print(f"Invalid geometry at index: {e}")
        
#         return ee_polygon

# # Function to process each sampling site and its dates
# def process_sampling_site(site):
#     # Get the sampling site id
#     seg_id = site['nzsegment']

#     # Convert the geometry to an ee.polygon
#     ee_polygon = geometry_to_ee_polygon(site['geometry'])

#     # Define list of the calculated indices
#     indices = ['MNDWI', 'NDVI', 'NIRv', 'NSMI', 'BSI', 'EVI', 'SAVI', 'NDMI', 'NBR', 'CI', 'LAI', 'FAPAR']

#     # Get the sampling dates for the current sampling site
#     sampeling_dates = filtered_stack_frame['myDate'][filtered_stack_frame['nzsegment'] == seg_id].unique()
    
#     dates_dict = {}
#     fixed_date = datetime.strptime("2020-12-03", "%Y-%m-%d").date()
#     for date in sampeling_dates:
#         if date > fixed_date:
#             try:
#                 START_DATE = (date - timedelta(days=14)).strftime("%Y-%m-%d")
#                 END_DATE = date.strftime("%Y-%m-%d")
#                 print('.....Date in progress:', START_DATE, END_DATE)
                
#                 # Create a cloud free sentinel-2 image for a specific region and time
#                 cld_free_image = cld_free_sl2(START_DATE, END_DATE, ee_polygon)
                
#                 # Calculated all RS indices and stack them into an EE Image.
#                 indices_image = calculate_indices(cld_free_image, indices, ee_polygon)

#                 # Calculate zonal stats
#                 stats_date_indices_dic = calculate_stats(indices_image, indices, ee_polygon)
       
#                 # Define the output file path
#                 stats_date_indices_dir = f'../Data/RS/RS_Indices_{seg_id}_{END_DATE}.p'
                
#                 # Save the dictionary as a pickle file
#                 with open(stats_date_indices_dir, 'wb') as f:
#                     pickle.dump(stats_date_indices_dic, f)

#                 # Add the stats result to the dictonary
#                 dates_dict[END_DATE]  = stats_date_indices_dic
            
#             except Exception as e:
#                 print(f"Something went wrong: {e}")
#                 continue
        
#     return dates_dict

# import os
# import geopandas as gpd
# from datetime import date, timedelta, datetime
# import pyproj
# from shapely.geometry import Polygon, MultiPolygon

# try:
#     import cPickle as pickle
# except ImportError:  # Python 3.x
#     import pickle

# # Read the pickle data
# in_f_dir = '../Data/WQ_Data_Nov2021.p'
# with open(in_f_dir, 'rb') as f:
#     dat_dic = pickle.load(f)

# # Prepare the sampling dataset
# stack_frame = dat_dic['StackFrame']
# metadata = dat_dic['metadataWQ'].drop_duplicates()

# # Filter the sampling dataset to contain only 2019 and later
# filtered_stack_frame = stack_frame[stack_frame['Year'] > 2018]
# filtered_stack_frame = filtered_stack_frame.reset_index(drop=True)
# filtered_metadata = metadata[metadata['nzsegment'].isin(filtered_stack_frame['nzsegment'])]
# filtered_metadata = filtered_metadata.reset_index(drop=True)

# # Initialize the empty dictionary for RS Indices
# rs_indices = {}

# # Process each sampling site using the function
# for i, site in filtered_metadata.iterrows():
#     # Get the sampling site id
#     seg_id = site['nzsegment']
#     print(f"{i}-", "We are collecting data for sampling site", seg_id)
    
#     # Get the sampling dates for the current sampling site
#     sampeling_dates = filtered_stack_frame['myDate'][filtered_stack_frame['nzsegment'] == seg_id].unique()
    
#     if i == 101:
#         print(i)            
#         rs_indices[site['nzsegment']] = process_sampling_site(site)

### Iterate over the observation dates in each sampling site

In [15]:
import sys
import subprocess
import pyproj

try:
    import ee
    print('The required modules are installed')
except ModuleNotFoundError:
    print('The required modules are NOT installed')

    # Install the required modules
    python = sys.executable
    subprocess.check_call(
        [python, '-m', 'pip', 'install', 'earthengine-api'],
        stdout=subprocess.DEVNULL
    )

## Initialize the library.
ee.Initialize()

# Function to perform zonal stats
def calculate_stats(image, indices, ee_polygon):    
    # Create an empty dictionary
    statistics_dict = {}
    
    # Calculate statistics for each index
    for index in indices:
        # Calculate multiple statistics using reduceRegion
        stats = image.reduceRegion(
            reducer=ee.Reducer.min().combine(
                reducer2=ee.Reducer.median().combine(
                    reducer2=ee.Reducer.max().combine(
                        reducer2=ee.Reducer.stdDev(),
                        sharedInputs=True
                    ),
                    sharedInputs=True
                ),
                sharedInputs=True
            ),
            geometry=ee_polygon,
            scale=10,
            maxPixels=1e19
        )
        
        # Add the statistics to the dictionary
        statistics_dict[index] = {
          # 'min': stats.get(f'{index}_min').getInfo(),
          'median': stats.get(f'{index}_median').getInfo(),
          # 'max': stats.get(f'{index}_max').getInfo(),
          'stdv': stats.get(f'{index}_stdDev').getInfo()
        }
        
    return statistics_dict

# Function to convert the geometry to an ee.polygon and transform the projection
def geometry_to_ee_polygon(geometry):
        # Transform the geometry to WGS84 projection
        source_crs = "EPSG:2193"
        target_crs = "EPSG:4326"
        transformer = pyproj.Transformer.from_crs(source_crs, target_crs, always_xy=True)
    
        # Reproject the polygon to the target CRS
        try:
            # Check if the geometery is a polygon or a multipolygon type
            if isinstance(geometry, MultiPolygon):
                geometry_list = []
                for polygon in geometry.geoms:
                    # Reproject the polygon to the target CRS
                    transformed_coordinates = [transformer.transform(x, y) for x, y in polygon.exterior.coords]
                    transformed_geometry = Polygon(transformed_coordinates)
                    
                    # Create an ee.Geometry.Polygon and add it to a list
                    ee_polygon = ee.Geometry.Polygon(transformed_coordinates)
                    geometry_list += [ee_polygon]
                    
                    # Create a ee.Geometry.MultiPolygon using the list of ee.Geometry objects
                    multi_polygon = ee.Geometry.MultiPolygon(geometry_list)
                    # Dissolve the ee.Geometry.MultiPolygon to create a single ee.Geometry.Polygon
                    ee_polygon = multi_polygon.dissolve()
            
            elif isinstance(geometry, Polygon):
                # Reproject the polygon to the target CRS
                transformed_coordinates = [transformer.transform(x, y) for x, y in geometry.exterior.coords]
                transformed_geometry = Polygon(transformed_coordinates)
                ee_polygon = ee.Geometry.Polygon(transformed_coordinates)
        
            else:
                raise ValueError("Invalid geometry type. Must be Polygon or MultiPolygon.")
        
        except Exception as e:
            print(f"Invalid geometry at index: {e}")
        
        return ee_polygon

# Function to process each sampling site and its dates
def process_sampling_site(site):
    # Get the sampling site id
    seg_id = site['nzsegment']
    print("We are collecting data for sampling site", seg_id)

    # Convert the geometry to an ee.polygon
    ee_polygon = geometry_to_ee_polygon(site['geometry'])

    # Define list of the calculated indices
    indices = ['MNDWI', 'NDVI', 'NIRv', 'NSMI', 'BSI', 'EVI', 'SAVI', 'NDMI', 'NBR', 'CI', 'LAI', 'FAPAR']

    # Get the sampling dates for the current sampling site
    sampeling_dates = filtered_stack_frame['myDate'][filtered_stack_frame['nzsegment'] == seg_id].unique()
    
    dates_dict = {}
    fixed_date = datetime.strptime("2017-12-03", "%Y-%m-%d").date()
    for date in sampeling_dates:
        if date > fixed_date:
            try:
                START_DATE = (date - timedelta(days=14)).strftime("%Y-%m-%d")
                END_DATE = date.strftime("%Y-%m-%d")
                print('.....Date in progress:', START_DATE, END_DATE)
                
                # Create a cloud free sentinel-2 image for a specific region and time
                cld_free_image = cld_free_sl2(START_DATE, END_DATE, ee_polygon)
                
                # Calculated all RS indices and stack them into an EE Image.
                indices_image = calculate_indices(cld_free_image, indices, ee_polygon)

                # Calculate zonal stats
                stats_date_indices_dic = calculate_stats(indices_image, indices, ee_polygon)
       
                # Define the output file path
                stats_date_indices_dir = f'../Data/RS/RS_Indices_{seg_id}_{END_DATE}.p'
                
                # Save the dictionary as a pickle file
                with open(stats_date_indices_dir, 'wb') as f:
                    pickle.dump(stats_date_indices_dic, f)

                # Add the stats result to the dictonary
                dates_dict[END_DATE]  = stats_date_indices_dic
            
            except Exception as e:
                print(f"Something went wrong: {e}")
                continue
        
    return dates_dict

# The function to process each sampling site and its dates
def process_sampling_site_wrapper(site):
    try:
        return process_sampling_site(site)
    except Exception as e:
        print(f"Error processing site {site['nzsegment']}: {e}")
        return None

The required modules are installed


In [ ]:
import concurrent.futures
import os
import geopandas as gpd
from datetime import date, timedelta, datetime
import pyproj
from shapely.geometry import Polygon, MultiPolygon

try:
    import cPickle as pickle
except ImportError:  # Python 3.x
    import pickle

# Read the pickle data
in_f_dir = '../Data/WQ_Data_Nov2021.p'
with open(in_f_dir, 'rb') as f:
    dat_dic = pickle.load(f)

# Prepare the sampling dataset
stack_frame = dat_dic['StackFrame']
metadata = dat_dic['metadataWQ'].drop_duplicates()

# Filter the sampling dataset to contain only 2019 and later
filtered_stack_frame = stack_frame[stack_frame['Year'] > 2018]
filtered_stack_frame = filtered_stack_frame.reset_index(drop=True)
filtered_metadata = metadata[metadata['nzsegment'].isin(filtered_stack_frame['nzsegment'])]
filtered_metadata = filtered_metadata.reset_index(drop=True)
# filtered_metadata = filtered_metadata.head(2)

# Initialize the empty dictionary for RS Indices
rs_indices = {}

# Process each sampling site using ThreadPoolExecutor for parallel processing
with concurrent.futures.ThreadPoolExecutor() as executor:
    futures = [executor.submit(process_sampling_site_wrapper, site) for i, site in filtered_metadata.iterrows()]
    for i, future in enumerate(concurrent.futures.as_completed(futures)):
        site_result = future.result()
        if site_result is not None:
            seg_id = filtered_metadata.loc[i, 'nzsegment']
            rs_indices[seg_id] = site_result
    rs_indices

We are collecting data for sampling siteWe are collecting data for sampling site 2036529.0
 2038450.0
We are collecting data for sampling site 2035811.0
We are collecting data for sampling site 2032466.0
We are collecting data for sampling site 2032082.0
We are collecting data for sampling site 2031444.0
.....Date in progress: 2019-01-01 2019-01-15
We are collecting data for sampling site 2045595.0
.....Date in progress: 2018-12-26 2019-01-09
We are collecting data for sampling site 2038228.0
.....Date in progress: 2018-12-31 2019-01-14
We are collecting data for sampling site 2038644.0
.....Date in progress: 2018-12-26 2019-01-09
We are collecting data for sampling site 2040298.0
We are collecting data for sampling site 2040105.0
We are collecting data for sampling site 2040035.0
.....Date in progress: 2018-12-31 2019-01-14
We are collecting data for sampling site 2039146.0
We are collecting data for sampling site 2039478.0
We are collecting data for sampling site 2041640.0
We are col

In [9]:
try:
    import cPickle as pickle
except ImportError:  # Python 3.x
    import pickle
    
# Read the pickle data
in_f_dir = '../Data/RS/RS_Indices_2032082.0_2020-12-07.p'
with open(in_f_dir, 'rb') as f:
    temp = pickle.load(f)

temp

{'MNDWI': {'median': -0.4570643609478675, 'stdv': 0.247108336037453},
 'NDVI': {'median': 0.8242632860550974, 'stdv': 0.2555919154041939},
 'NIRv': {'median': 2095.950150618131, 'stdv': 1016.5055304979722},
 'NSMI': {'median': 0.41800229193754873, 'stdv': 0.18830273635057188},
 'BSI': {'median': -0.7090008578954345, 'stdv': 0.09439932792387523},
 'EVI': {'median': 2.223268182950829, 'stdv': 8.604207341983171},
 'SAVI': {'median': 1.2267477643142597, 'stdv': 0.38329392084271563},
 'NDMI': {'median': 0.4180022919375486, 'stdv': 0.1883027363505718},
 'NBR': {'median': 0.6758377966287344, 'stdv': 0.21276439949672504},
 'CI': {'median': -0.39072746716167983, 'stdv': 0.18059132819261745},
 'LAI': {'median': 1.6191469286869756e+266, 'stdv': 'NaN'},
 'FAPAR': {'median': 1.031805197549263, 'stdv': 0.3084801189969154}}

## Iterate over each observation time and collect the satelite indices for all sampling sites

### Calculate a list of indices for a specific site and return the image

In [ ]:
import ee

# Initialize Earth Engine
ee.Initialize()

# Function to calculate a list of indices for a specific site and return the image
def calculate_indices(img, indices, ee_polygon):
    # Initialize an empty ee.Image to store the calculated indices
    calc_indices = ee.Image()
    
    # Iterate over the list of indices and calculate each requested index in the list
    for index in indices:
        if 'MNDWI' in indices: # Modified Normalized Difference Water Index     
            mndwi = img.normalizedDifference(['B3','B11']).rename("MNDWI") 
            calc_indices = calc_indices.addBands(mndwi)
            
        elif 'NDVI' in indices: # Normalized Difference Vegetation Index
            ndvi = img.normalizedDifference(['B8','B4']).rename("NDVI")
            calc_indices = calc_indices.addBands(ndvi)
            
        elif 'NIRv': # Another vegetation index
            ndvi = img.normalizedDifference(['B8','B4']).rename("NDVI") 
            nir = img.select('B8')
            nirv = ndvi.multiply(nir).rename("NIRv")
            calc_indices = calc_indices.addBands(nirv)
            
        elif 'NSMI' in indices: # Normalized Soil Index
            nsmi = img.normalizedDifference(['B8','B11']).rename("NSMI")
            calc_indices = calc_indices.addBands(nsmi)
            
        elif 'BSI' in indices: # Bare Soil Index
            # Calculate the Bare Soil Index (BSI)
            bsi = img.normalizedDifference(['B3', 'B11'])\
                    .subtract(img.select('B8')).divide(img.select('B11')\
                                                       .add(img.select('B8')))
            # Rename the BSI band
            bsi = bsi.rename("BSI")
            calc_indices = calc_indices.addBands(bsi)
            
        elif 'EVI' in indices: # Enhanced Vegetation Index
            evi = img.expression('2.5 * ((NIR - RED) / (NIR + 6 * RED - 7.5 * BLUE + 1))',\
                                 {'NIR': img.select('B8'), 'RED': img.select('B4'),\
                                  'BLUE': img.select('B2')}).rename("EVI")
            calc_indices = calc_indices.addBands(evi)
        
        elif 'SAVI' in indices: # Soil Adjusted Vegetation Index
            savi = img.expression('(1 + 0.5) * (NIR - RED) / (NIR + RED + 0.5)',\
                                  {'NIR': img.select('B8'), 'RED': img.select('B4')})\
                                   .rename("SAVI")
            calc_indices = calc_indices.addBands(savi)
            
        elif 'NDMI' in indices: # Normalized Difference Moisture Index
            ndmi = img.normalizedDifference(['B8', 'B11']).rename("NDMI")
            calc_indices = calc_indices.addBands(ndmi)
            
        elif 'NBR' in indices: # Normalized Burn Ratio
            nbr = img.normalizedDifference(['B8', 'B12']).rename("NBR")
            calc_indices = calc_indices.addBands(nbr)
            
        elif 'CI' in indices: # Chlorophyll Index
            ci = img.select('B4').divide(img.select('B3')).subtract(1).rename("CI")
            calc_indices = calc_indices.addBands(ci)
            
        elif 'LAI' in indices: # Leaf Area Index
            lai = img.expression('3.618 * exp(-0.488 * (RED - GREEN))',\
                                 {'RED': img.select('B4'), 'GREEN': img.select('B3')}).rename("LAI")
            calc_indices = calc_indices.addBands(lai)
            
        elif 'FAPAR' in indices: # Fraction of Absorbed Photosynthetically Active Radiation
            fapar = img.expression('(NIR - RED) / (0.88 * NIR + 0.12 * RED)',\
                                   {'NIR': img.select('B8'), 'RED': img.select('B4')}).rename("FAPAR")
            calc_indices = calc_indices.addBands(fapar)

    # # Copy the properties from img to calc_indices to ensure the same metadata
    # calc_indices = calc_indices.copyProperties(img)
    
    # Clip the index image to the extent of the current sampling site catchment
    calc_indices = calc_indices.clip(ee_polygon)

    return calc_indices

### Claculate Zonal Statistics

In [ ]:
import ee

# Initialize Earth Engine
ee.Initialize()

def calculate_stats(image, feature):
    # Get the geometry of the feature
    geom = feature.geometry()
    
    # Reduce the image within the polygon to get median and stdDev
    reducer = ee.Reducer.median().combine(ee.Reducer.stdDev(), '', True)
    stats = image.reduceRegion(reducer, geom, 30)
    
    # Extract the median and stdDev values from the computed dictionary
    median_vals = [stats.get(f'{band}_median') for band in image.bandNames().getInfo()]
    std_dev_vals = [stats.get(f'{band}_stdDev') for band in image.bandNames().getInfo()]
    
    # Create a dictionary containing band names and their corresponding median and stdDev values
    band_stats = dict(zip(image.bandNames().getInfo(), zip(median_vals, std_dev_vals)))
    
    # Return the dictionary along with the feature's properties
    return feature.set(band_stats)

### Read the data

In [ ]:
print("Loading required libraries ....")
# Import necessary libraries
import os
import geopandas as gpd
from datetime import date, timedelta, datetime
import pyproj
from shapely.geometry import Polygon, MultiPolygon

try:
  import cPickle as pickle
except ImportError:  # Python 3.x
  import pickle

# Read the pickle data
in_f_dir = '../Data/WQ_Data_Nov2021.p'
with open(in_f_dir, 'rb') as f:
    dat_dic = pickle.load(f)
print("All required libraries were successfully added")

print("Loading the sampling dataset ....")
# Prepare the sampling dataset
stack_frame = dat_dic['StackFrame']
metadata = dat_dic['metadataWQ'].drop_duplicates()
print("The sampeling dataset was successfully loaded")

print("Preparing the samppeling dataset")
# Filter the sampling dataset to contain only 2019 and later
filtered_stack_frame = stack_frame[stack_frame['Year'] > 2018]
filtered_stack_frame = filtered_stack_frame.reset_index(drop=True)
filtered_metadata = metadata[metadata['nzsegment'].isin(filtered_stack_frame['nzsegment'])]
filtered_metadata = filtered_metadata.reset_index(drop=True)

print("Loading the sampling site polygons ...")
# Load the shapefile using geopandas
shapefile_path = '../Data/SamplingSites_CZ_Topo.shp'
# shapefile = gpd.read_file(shapefile_path)
print("The sampling site polygons was successfully loaded")

print("Converting the sampling site polygons shapefile into ee.FeatureCollection ...")
# Convert Polygon Shapefile into ee.FeatureCollection 
feature_collection = shapefile_2_ee_collection(shapefile)
print("The sampling site polygons shapefile was successfully converted into ee.FeatureCollection")

# Define the bounding box for New Zealand [west, south, east, north]
new_zealand_geometry = ee.Geometry.Rectangle([165.0, -47.5, 179.0, -34.0])

print("Getting the sampling times ...")
# Get the sampling dates
sampeling_dates = filtered_stack_frame['myDate'].unique()

### Process the data

#### Single computation

In [ ]:
# # Define list of the calculated indices
# indices = ['MNDWI', 'NDVI', 'NIRv', 'NSMI', 'BSI', 'EVI', 'SAVI', 'NDMI', 'NBR', 'CI', 'LAI', 'FAPAR']

# date_dict = {}
# for date in sampeling_dates:
#     try:
#         START_DATE = (date - timedelta(days=14)).strftime("%Y-%m-%d")
#         END_DATE = date.strftime("%Y-%m-%d")
#         print("The Image for Date :", START_DATE, END_DATE, "is getting collected from Google Earth Engine")
        
#         # Create a cloud free sentinel-2 image for a specific region and time
#         cld_free_image =  cld_free_sl2(START_DATE, END_DATE, new_zealand_geometry)
        
#         # Calculate a list of indices for a specific site and return the image
#         indices_img = calculate_indices(cld_free_image, indices, new_zealand_geometry)
        
        
#         # Calculate the statistics for each polygon in the feature_collection
#         stats_by_polygon = feature_collection.map(lambda feature: calculate_stats(indices_img, feature))
        
#         # Print the result
#         print(stats_by_polygon.getInfo())
    
#     except Exception as e:
#         print(f"Something went wrong: {e}")
#         continue

#### Batch Computation

In [ ]:
# # Define list of the calculated indices
# indices = ['MNDWI', 'NDVI', 'NIRv', 'NSMI', 'BSI', 'EVI', 'SAVI', 'NDMI', 'NBR', 'CI', 'LAI', 'FAPAR']

# # Define the batch size (number of features per batch)
# batch_size = 50
# # Split the feature_collection into smaller batches
# feature_batches = []
# num_features = feature_collection.size().getInfo()
# for i in range(0, num_features, batch_size):
#     batch = feature_collection.toList(batch_size, i)
#     feature_batches.append(ee.FeatureCollection(batch))

# print("Calculating Zonal Stats for each date ...")
# date_dict = {}
# # Iterate over the sampling dates and calculate the RS indices for all polygons in each date
# for date in sampeling_dates:
#     START_DATE = (date - timedelta(days=14)).strftime("%Y-%m-%d")
#     END_DATE = date.strftime("%Y-%m-%d")
#     print("The Image for Date :", START_DATE, END_DATE, "is getting collected from Google Earth Engine")

#     # Process each batch
#     for batch in feature_batches:
#         try:
#             # Create a cloud free sentinel-2 image for a specific region and time
#             cld_free_image = cld_free_sl2(START_DATE, END_DATE, new_zealand_geometry)
            
#             # Calculate a list of indices for a specific site and return the image
#             indices_img = calculate_indices(cld_free_image, indices, new_zealand_geometry)
            
#             # Calculate the statistics for each polygon in the batch
#             stats_by_polygon = batch.map(lambda feature: calculate_stats(indices_img, feature))
            
#             # Print the result for this batch
#             print(stats_by_polygon.getInfo())
        
#         except Exception as e:
#             print(f"Something went wrong: {e}")
#             continue